In [2]:
from functools import wraps
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from shared.ffmpegw import export_frames_as_gif, export_frames_as_mp4
from shared.pathutils import init_folder

class FunctionGenerator:

  class _Base:
    atol = 1e-5

    @classmethod
    @wraps(np.sin)
    def sin(cls, s, *args, **kwargs):
      s = np.sin(s, *args, **kwargs)
      if isinstance(s, np.ndarray):
        s[np.isclose(s, 0, atol=cls.atol)] = 0
      else:
        if np.isclose(s, 0, atol=cls.atol):
          s = 0
      return s

    def __init__(self, frequency, amplitude=1, phase=0):
      self.w = 0
      self.f = frequency
      self.a = amplitude
      self.p = phase

    @property
    def f(self):
      return self.w / np.pi / 2

    @f.setter
    def f(self, frequency):
      self.w = frequency * np.pi * 2


  class Sine(_Base):
    def S(self, samples):
      return self.a * self.sin(samples * self.w + self.p)


  class Square(_Base):
    def S(self, samples):
      s = self.sin(samples * self.w + self.p)
      return self.a * np.sign(s)


  class Sawtooth(_Base):
    def S(self, samples):
      p = 1 / self.f
      t = self.p / np.pi / 2
      s = samples + t / p
      s -= np.floor(.5 + ((samples + t) / p))
      s *= 2
      return self.a * s


  class Triangle(_Base):
    def S(self, samples):
      p = 1 / self.f
      t = (self.p + np.pi/2) / np.pi / 2
      s = (samples + t) / p
      s -= np.floor(.5 + ((samples + t) / p))
      s *= 2
      s = np.abs(s)
      s *= 2
      s -= 1
      return self.a * s


  class _FSEBase(_Base):
    def kw(self, i):
      raise NotImplementedError

    def ka(self, i):
      raise NotImplementedError

    def t(self, samples, i):
      return samples * self.w * self.kw(i) + self.p

    def s(self, samples, i):
      return self.sin(self.t(samples, i)) * self.ka(i) * self.a

    def S(self, samples, n_iter=10):
      return np.sum([self.s(samples, i+1) for i in range(n_iter)], axis=0)


  class FSESine(_FSEBase):

    @staticmethod
    def kw(i):
      return 1

    @classmethod
    def ka(cls, i):
      return 1

    def S(self, samples, n_iter=10):
      return self.s(samples, 1)


  class FSESquare(_FSEBase):

    @staticmethod
    def kw(i):
      k = (i * 2) - 1
      return k

    @classmethod
    def ka(cls, i):
      k = 1 / cls.kw(i)
      k *= 4 / np.pi
      return k


  class FSESawtooth(_FSEBase):

    @staticmethod
    def kw(i):
      return i

    @classmethod
    def ka(cls, i):
      k = (-1) ** i
      k /= i
      k *= -2 / np.pi
      return k


  class FSETriangle(_FSEBase):
    pi2 = np.pi**2

    @staticmethod
    def kw(i):
      return i * 2 - 1

    @classmethod
    def ka(cls, i):
      k = (-1) ** i
      k /= cls.kw(i) ** 2
      k *= -8 / cls.pi2
      return k


In [23]:
N_FRAMES = 125
FPS = 25
N_SAMPLES = 1001
RANGE_SAMPLES = (0, 1)
N_MODES = 10
FREQUENCY = 1
FUNCTIONS = [
  (FunctionGenerator.Square, FunctionGenerator.FSESquare),
  (FunctionGenerator.Sawtooth, FunctionGenerator.FSESawtooth),
  (FunctionGenerator.Triangle, FunctionGenerator.FSETriangle),
]

samples = np.linspace(*RANGE_SAMPLES, N_SAMPLES)
iframes = np.linspace(0, N_SAMPLES-1, N_FRAMES).astype(np.int32)
gs = mpl.gridspec.GridSpec(len(FUNCTIONS), 2, width_ratios=[1, 3])
buf_lin = np.zeros((len(FUNCTIONS), N_SAMPLES))
buf_fse = np.zeros((len(FUNCTIONS), N_MODES, N_SAMPLES))

init_folder('./frames/funcgen3')

for n, (gen_lin, _) in enumerate(FUNCTIONS):
  buf_lin[n, :] += gen_lin(FREQUENCY).S(samples)

for m in range(N_MODES):
  for n, (_, gen_fse) in enumerate(FUNCTIONS):
    g = gen_fse(FREQUENCY)
    buf_fse[n, m, :] += g.s(samples, m+1)

for f in range(N_FRAMES):
  i = iframes[f]

  fig = plt.figure(figsize=(10, 4*len(FUNCTIONS)))
  for n in range(len(FUNCTIONS)):
    ax1 = fig.add_subplot(gs[n*2])
    harmonics = np.cumsum(buf_fse[n, :, i])
    for m in range(N_MODES):
      harmonic_max = abs(FUNCTIONS[n][1].ka(m+1))
      harmonic_curr = harmonics[m]
      harmonic_prev = 0 if m-1 < 0 else harmonics[m-1]
      ax1.scatter(m, harmonic_curr)
      ax1.plot((m, m, m+1), (harmonic_prev, harmonic_curr, harmonic_curr))
      ax1.plot((m, m), (harmonic_prev-harmonic_max, harmonic_prev+harmonic_max), linestyle=':', c='gray')
    ax1.set_xlim(-.5, N_MODES)
    ax1.set_ylim(-1.30, 1.30)
    ax1.set_xticks(range(N_MODES), map(lambda j: f'{int(FUNCTIONS[n][1].kw(j+1)*np.sign(FUNCTIONS[n][1].ka(j+1)))}w {abs(FUNCTIONS[n][1].ka(j+1)):.03f}A', range(N_MODES)), rotation='vertical')

    ax2 = fig.add_subplot(gs[n*2+1])
    ax2.plot(samples, buf_lin[n, :], c='black', linestyle=':')
    ax2.plot(samples, buf_fse[n, :N_MODES, :].sum(axis=0))
    y = buf_fse[n, :N_MODES, i].sum(axis=0)
    ax2.plot((samples[0], samples[i]), (y, y), c='grey', linestyle='-')
    ax2.scatter(samples[i], y, c='black')

    ax2.sharey(ax1)
    ax2.get_yaxis().set_visible(False)
    ax2.set_xlim(samples[0], samples[-1])

  fig.tight_layout()
  fig.savefig(f'./frames/funcgen3/frame_{f:04d}.webp')
  plt.close()

export_frames_as_gif('./frames/funcgen3', 'frame_', './exports/funcgen3.gif', FPS)
